In [37]:
import torch
import os
import pandas as pd

testing = True
# freeze_encoder = args.freeze_encoder
GPU_NO = int('1')
model_ckpt_path = None

print(f'Test mode: {testing}')
# print(f'Freeze encoder: {freeze_encoder}')
os.environ["CUDA_VISIBLE_DEVICES"] = str(GPU_NO)  # use the correct gpu
device = torch.device(f'cuda' if torch.cuda.is_available() else 'cpu')
print(f"device: {device}")
if model_ckpt_path:
    print(f"model checkpoint to use: {model_ckpt_path}")
else:
    print("no ckpt specified. Finetuning from base")
# print(f"memory reserved: {torch.cuda.memory_reserved(device=device)} bytes")

root = '/home/ugrads/a/aa_ron_su/physionet.org/files/clinical-t5/1.0.0/'
data_path = '/data/datasets/mimiciv_notes/physionet.org/files/mimic-iv-note/2.2/note/radiology.csv'
model_path = root + 'Clinical-T5-Base/'
temivef_train_NOTE_TARGET1_FT_path = '/home/ugrads/a/aa_ron_su/JSS_SUBMISSION_NEW/data/till_end_mimic_iv_extra_features_train_NOTE_TARGET1_FT_rad.csv'

out_dir = f"{os.environ.get('CLINICAL_DIR')}model{'_test' if testing else ''}{'_from_ckpt1' if model_ckpt_path else ''}"
if not os.path.exists(out_dir):
    os.makedirs(out_dir)
model_out = os.path.join(out_dir,"meta_ft_classify.pt")
print(f"model_out:{model_out}")

from transformers import AutoModelForSeq2SeqLM, AutoTokenizer
model_name = "Clinical-T5-Base"
tokenizer = AutoTokenizer.from_pretrained("Clinical-T5-Base")
model = AutoModelForSeq2SeqLM.from_pretrained("Clinical-T5-Base")

train = pd.read_csv(temivef_train_NOTE_TARGET1_FT_path)
print(f"reading notes and target from {temivef_train_NOTE_TARGET1_FT_path}")

train_tensor_filename = f"{os.environ.get('CLINICAL_DIR')}tokenized_notes_rad/train_tensor.pt"
train_tensors = None
if os.path.isfile(train_tensor_filename):
    train_tensors = torch.load(train_tensor_filename)
    print(f"tokenized note input_ids loaded from {train_tensor_filename}")
# else:
#     train_texts = train['text'].tolist()
#     tokenized_train_notes = tokenizer(train_texts, truncation=True, padding=True, return_tensors = "pt")
#     train_tensor = tokenized_train_notes.input_ids
#     torch.save(train_tensor, train_tensor_filename)
#     print(f"train notes tokenized and saved to {train_tensor_filename}")
# test_texts = test['text'].tolist()
# tokenized_test_notes = tokenizer(test_texts, truncation=True, padding=True, return_tensors = "pt")
# print("test notes tokenized")


from T5EncoderForSequenceClassification import T5EncoderForSequenceClassification

from transformers import T5Config

encoder = model.get_encoder() # we only need the clinical-t5 encoder for our purposes

config = T5Config(
    hidden_size=768,
    classifier_dropout=None,
    num_labels=2,
    hidden_dropout_prob=0.01,
    last_hidden_size=64
)

classifier = (
    T5EncoderForSequenceClassification(encoder, config).to(device) 
    if model_ckpt_path == None 
    else torch.load(model_ckpt_path)
)

print(f"CLASSIFIER LOADED TO GPU: {torch.cuda.memory_allocated() / 2**20} Megabytes")

from torch.utils.data import DataLoader, TensorDataset, RandomSampler
from transformers import get_linear_schedule_with_warmup
import numpy as np

# Define training data
train_inputs = train_tensors.input_ids.to(device)
train_labels = torch.tensor(train['delta_in_2_days'].to_numpy()).to(device)
train_dataset = TensorDataset(train_inputs, train_labels) 

Test mode: True
device: cuda
no ckpt specified. Finetuning from base
model_out:/home/ugrads/a/aa_ron_su/physionet.org/files/clinical-t5/1.0.0/model_test/meta_ft_classify.pt
reading notes and target from /home/ugrads/a/aa_ron_su/JSS_SUBMISSION_NEW/data/till_end_mimic_iv_extra_features_train_NOTE_TARGET1_FT_rad.csv
tokenized note input_ids loaded from /home/ugrads/a/aa_ron_su/physionet.org/files/clinical-t5/1.0.0/tokenized_notes_rad/train_tensor.pt
CLASSIFIER LOADED TO GPU: 841.216796875 Megabytes


In [40]:
len(train_dataset)

25

In [38]:
train_inputs

tensor([[  262,     4,  4815,  ...,     0,     0,     0],
        [ 3388, 27447,  8015,  ...,     0,     0,     0],
        [    3,  7094, 22633,  ...,     0,     0,     0],
        ...,
        [    3, 12016,  5668,  ...,     0,     0,     0],
        [  262,     4,   188,  ...,     0,     0,     0],
        [ 3388, 27447,  8015,  ...,     0,     0,     0]], device='cuda:0')

In [ ]:
train_tensors

{'input_ids': tensor([[  262,     4,   188, 17684,  8015,    10,  9302,  6038,    41, 14536,
         17098,     3,  2965,    61,  3388, 27447,  8015,    10,     3,   834,
           834,   834,    28, 19222, 13751, 12498,   433,    58, 23666, 22410,
         10466,    10,  4004,   222,  2252,  9413,     3,   834,   834,   834,
           377, 13885,  2365,   134,    10,  7871,   851,   138,   903,    13,
             8,  5738,   937,     5,   290,    19,   150, 15949, 16690,     6,
         13577, 11733,     6,    42, 18919,  8888, 21783,   226,     5,    37,
         16216,  8172,     7,    17, 10270, 19561,    19,  1389,     5,  7204,
           894,    33,  1317, 16234,   516,    53,   147,     8,   646,  6748,
            11,  4322,   646,    18, 17389,     3,  6520, 16898,     7,     5,
           465,   339,   799,   666,     8,   269,     3,    88, 16091,     9,
         28698,   122,    51,    19,   894,     5,     3,  5166, 22120,   134,
          9215,    10,   465, 12498, 1

In [15]:
import pandas as pd
import torch
import os

temivef_train_NOTE_TARGET1_FT_path = '/home/ugrads/a/aa_ron_su/JSS_SUBMISSION_NEW/data/till_end_mimic_iv_extra_features_train_NOTE_TARGET1_FT_rad.csv'
temivef_test_NOTE_TARGET1_FT_path = '/home/ugrads/a/aa_ron_su/JSS_SUBMISSION_NEW/data/till_end_mimic_iv_extra_features_test_NOTE_TARGET1_FT_rad.csv'

from transformers import AutoModelForSeq2SeqLM, AutoTokenizer
model_name = "Clinical-T5-Base"
tokenizer = AutoTokenizer.from_pretrained("Clinical-T5-Base")

test = pd.read_csv(temivef_test_NOTE_TARGET1_FT_path)
print(f"reading notes and target from {temivef_test_NOTE_TARGET1_FT_path}")

train = pd.read_csv(temivef_train_NOTE_TARGET1_FT_path)
print(f"reading notes and target from {temivef_train_NOTE_TARGET1_FT_path}")

print('CLINICAL_DIR:', os.environ.get('CLINICAL_DIR'))
tensor_dir = os.path.join(os.environ.get('CLINICAL_DIR'), "tokenized_notes_rad")
train_tensor_path = os.path.join(tensor_dir, "train_tensor.pt")
test_tensor_path = os.path.join(tensor_dir, "test_tensor.pt")

if not os.path.exists(tensor_dir):
    os.makedirs(tensor_dir)


train_texts = train['text'].tolist()
tokenized_train_notes = tokenizer(train_texts, truncation=True, padding=True, return_tensors = "pt")
train_tensors = tokenized_train_notes
torch.save(train_tensors, train_tensor_path)
print(f"train notes tokenized and saved to {train_tensor_path}")


test_texts = test['text'].tolist()
tokenized_test_notes = tokenizer(test_texts, truncation=True, padding=True, return_tensors = "pt")
test_tensor = tokenized_test_notes
torch.save(test_tensor, test_tensor_path)
print(f"test notes tokenized and saved to {test_tensor_path}")

reading notes and target from /home/ugrads/a/aa_ron_su/JSS_SUBMISSION_NEW/data/till_end_mimic_iv_extra_features_test_NOTE_TARGET1_FT_rad.csv
reading notes and target from /home/ugrads/a/aa_ron_su/JSS_SUBMISSION_NEW/data/till_end_mimic_iv_extra_features_train_NOTE_TARGET1_FT_rad.csv
CLINICAL_DIR: /home/ugrads/a/aa_ron_su/physionet.org/files/clinical-t5/1.0.0/
train notes tokenized and saved to /home/ugrads/a/aa_ron_su/physionet.org/files/clinical-t5/1.0.0/tokenized_notes_rad/train_tensor.pt
test notes tokenized and saved to /home/ugrads/a/aa_ron_su/physionet.org/files/clinical-t5/1.0.0/tokenized_notes_rad/test_tensor.pt


In [18]:
train_tensors

{'input_ids': tensor([[  262,     4,   188, 17684,  8015,    10,  9302,  6038,    41, 14536,
         17098,     3,  2965,    61,  3388, 27447,  8015,    10,     3,   834,
           834,   834,    28, 19222, 13751, 12498,   433,    58, 23666, 22410,
         10466,    10,  4004,   222,  2252,  9413,     3,   834,   834,   834,
           377, 13885,  2365,   134,    10,  7871,   851,   138,   903,    13,
             8,  5738,   937,     5,   290,    19,   150, 15949, 16690,     6,
         13577, 11733,     6,    42, 18919,  8888, 21783,   226,     5,    37,
         16216,  8172,     7,    17, 10270, 19561,    19,  1389,     5,  7204,
           894,    33,  1317, 16234,   516,    53,   147,     8,   646,  6748,
            11,  4322,   646,    18, 17389,     3,  6520, 16898,     7,     5,
           465,   339,   799,   666,     8,   269,     3,    88, 16091,     9,
         28698,   122,    51,    19,   894,     5,     3,  5166, 22120,   134,
          9215,    10,   465, 12498, 1

In [22]:
import os
import torch
import argparse
import pandas as pd


testing = True
GPU_NO = 4
print(f'Test mode: {testing}')
os.environ["CUDA_VISIBLE_DEVICES"] = str(GPU_NO)  # use the correct gpu
device = torch.device(f'cuda' if torch.cuda.is_available() else 'cpu')
print(f"device: {device}")


# import torch.nn as nn
# import sys

root = '/home/ugrads/a/aa_ron_su/physionet.org/files/clinical-t5/1.0.0'
finetuned_model_path = root + '/model_from_ckpt1/meta_ft_classify.pt' # modify this line!
temivef_train_NOTE_TARGET1_FT_path = '/home/ugrads/a/aa_ron_su/JSS_SUBMISSION_NEW/data/till_end_mimic_iv_extra_features_train_NOTE_TARGET1_FT.csv'
temivef_test_NOTE_TARGET1_FT_path = '/home/ugrads/a/aa_ron_su/JSS_SUBMISSION_NEW/data/till_end_mimic_iv_extra_features_test_NOTE_TARGET1_FT.csv'
temivef_train_NOTE_path = '/home/ugrads/a/aa_ron_su/JSS_SUBMISSION_NEW/data/till_end_mimic_iv_extra_features_train_NOTE.csv'
temivef_test_NOTE_path = '/home/ugrads/a/aa_ron_su/JSS_SUBMISSION_NEW/data/till_end_mimic_iv_extra_features_test_NOTE.csv'
outdir = f'/home/ugrads/a/aa_ron_su/JSS_SUBMISSION_NEW/data/final_rad1_{"test" if testing else ""}/' # modify this line!
train_outpath = os.path.join(outdir, 'till_end_mimic_iv_extra_features_train.csv')
test_outpath = os.path.join(outdir, 'till_end_mimic_iv_extra_features_test.csv') 
from transformers import AutoTokenizer
model_name = "Clinical-T5-Base"
tokenizer = AutoTokenizer.from_pretrained("Clinical-T5-Base")

if not os.path.exists(outdir):
    os.makedirs(outdir)

tensor_dir = f"{os.environ.get('CLINICAL_DIR')}tokenized_notes_rad"
train_tokenizer_outputs_path = os.path.join(tensor_dir, "train_tensor.pt")
test_tokenizer_outputs_path = os.path.join(tensor_dir, "test_tensor.pt")

# if not os.path.exists(tensor_dir):
    # notes_to_extract = temivef_train_NOTE_TARGET1_FT_path['text']
    # texts = notes_to_extract.tolist()
    # tokenized_notes = tokenizer(texts, truncation=True, padding=True, return_tensors = "pt")
    # torch.save(tokenized_notes, train_tokenizer_outputs_path)

    # notes_to_extract = temivef_test_NOTE_TARGET1_FT_path['text']
    # texts = notes_to_extract.tolist()
    # tokenized_notes = tokenizer(texts, truncation=True, padding=True, return_tensors = "pt")
    # torch.save(tokenized_notes, test_tokenizer_outputs_path)

tokenized_train_notes = torch.load(train_tokenizer_outputs_path)
tokenized_test_notes = torch.load(test_tokenizer_outputs_path)

# if testing:
#     tokenized_train_notes = tokenized_train_notes[:12]
#     tokenized_test_notes = tokenized_test_notes[:12]
#     print(f"testing mode truncated tokenized_notes to length {len(tokenized_train_notes)}")


from torch.utils.data import DataLoader, TensorDataset
from time import time
import numpy as np
import torch

def generate_dataloader(tokenized_notes, batch_size, device):
    labels = torch.tensor([-1] * len(tokenized_notes.input_ids)) # not actually used in this case, since we are not evaluating loss
    dataset = TensorDataset(tokenized_notes.input_ids.to(device), tokenized_notes.attention_mask.to(device), labels.to(device))
    dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=False, drop_last=False) 
    # shuffle is false so that notes retain their order for concat with df
    return dataloader

batch_size = 6
train_dataloader = generate_dataloader(tokenized_train_notes, batch_size, device)
test_dataloader = generate_dataloader(tokenized_test_notes, batch_size, device)
print("dataloaders generated")

Test mode: True
device: cuda
dataloaders generated


In [28]:
with open('tmp.txt', 'w') as f:

    # Iterate over the batches in the dataloader
    for batch in train_dataloader:
        input_ids_batch, attention_mask_batch, labels_batch = batch

        # Write the contents of the batch to the file
        f.write("Input IDs batch: {}\n".format(input_ids_batch))
        f.write("Attention mask batch: {}\n".format(attention_mask_batch))
        f.write("Labels batch: {}\n".format(labels_batch))
        f.write("\n")

In [24]:
classifier = torch.load(finetuned_model_path)
classifier.encoder.eval() # makes sure dropout does not occur
classifier.classifier.eval()
print(f"loaded classifier from {finetuned_model_path}")

def extract_embeddings(dataloader, classifier):
    with(torch.no_grad()):
        start_time = time()
        embeddings = []
        for step, batch in enumerate(dataloader):
                input_ids, attention_mask, _ = batch
                emb = classifier.forward(input_ids, attention_mask=attention_mask, return_embeddings=True)
                # embeddings.append(emb)
                # print(f"Step {step}/{len(dataloader)} | Time {time() - start_time : .2f} seconds")

    # embeddings = torch.cat(embeddings, dim=0)
    # embeddings = embeddings.cpu()
    # embeddings_df = pd.DataFrame(embeddings.detach().numpy()).add_prefix('emb')
    # print(f"Extracted {len(embeddings_df)} note embeddings. Shape: {embeddings_df.shape}") # should be size 64
    # return embeddings_df

train_embeddings_df = extract_embeddings(train_dataloader, classifier)
print("finished train embedding extraction") 
test_embeddings_df = extract_embeddings(test_dataloader, classifier)
print("finished test embedding extraction")


# for mode in ["train", "test"]:
#     print(f"reading from temivef_{mode}_NOTE_TARGET1_FT_path")
# train = pd.read_csv(temivef_train_NOTE_TARGET1_FT_path)
# test = pd.read_csv(temivef_test_NOTE_TARGET1_FT_path)
# print(f"loaded train notes to extract from {temivef_train_NOTE_TARGET1_FT_path}")
# print(f"loaded test notes to extract from {temivef_test_NOTE_TARGET1_FT_path}")

# #concat notes with ICUSTAY column for merging later
# train_df_small = pd.concat([train[['ICUSTAY_ID', 'NOTE_ID']], train_embeddings_df], axis = 1)
# test_df_small = pd.concat([test[['ICUSTAY_ID', 'NOTE_ID']], test_embeddings_df], axis = 1)

# print(f"concatenating train and train_embeddings_df with shape {train.shape} and {train_embeddings_df.shape} respectively")
# print(f"concatenating test and test_embeddings_df with shape {test.shape} and {test_embeddings_df.shape} respectively")


# for mode in ["train", "test"]:
#     print(f"reading from temivef_{mode}_NOTE_path")
# train_df_big = pd.read_csv(temivef_train_NOTE_path)
# test_df_big = pd.read_csv(temivef_test_NOTE_path)
# print(f"loaded train df to merge with from {temivef_train_NOTE_path}")
# print(f"loaded test notes to merge with from {temivef_test_NOTE_path}")


# def merge_and_fill_embeddings(df_small, df_big):
#     print(f"BEFORE merge: len = {len(df_big)}")
#     out_df = df_big.merge(df_small, on = ['ICUSTAY_ID','NOTE_ID'], how = 'left')
#     print(f"AFTER merge: len = {len(out_df)}")
#     def fill_embedding_na(note_id_group):
#         note_id_group = note_id_group.fillna(method='ffill').fillna(method='bfill')
#         return note_id_group

#     emb_cols = ['emb' + str(i) for i in range(64)]

#     print(f"BEFORE fill na: len = {len(out_df)}")
#     out_df[emb_cols] = out_df.groupby('NOTE_ID')[emb_cols].transform(fill_embedding_na) # transform preserves the shape of the original
#     print(f"AFTER fill na: len = {len(out_df)}")


#     print(f"copied {len(df_small)} embeddings into rows with the correct NOTE_ID")
#     print(f"len(out_df): {len(out_df)}")
#     print(f"No. nonnull embedding rows in out_df: {len(out_df[pd.notna(out_df['emb0'])])}")
#     return out_df

# train_out_df = merge_and_fill_embeddings(train_df_small, train_df_big)
# print("merged and filled embeddings for train_out_df")
# test_out_df = merge_and_fill_embeddings(test_df_small, test_df_big)
# print("merged and filled embeddings for test_out_df")


# # format like mimic_iv_train
# def format_cols(df):
#     df.drop(['text', 'NOTE_ID', 't_start_DT','INTIME'], axis=1, inplace=True)
#     df.rename(columns = {
#         'SUBJECT_ID':'subject',
#         'ICUSTAY_ID':'Icustay'
#         }, inplace = True)
#     return df

# train_out_df = format_cols(train_out_df)
# test_out_df = format_cols(test_out_df)

loaded classifier from /home/ugrads/a/aa_ron_su/physionet.org/files/clinical-t5/1.0.0/model_from_ckpt1/meta_ft_classify.pt
finished train embedding extraction
finished test embedding extraction


In [2]:
tokenized_test_notes

{'input_ids': tensor([[ 3388, 27447,  8015,  ...,     0,     0,     0],
        [ 8668,     3,  5359,  ...,    12,    36,     1],
        [ 9302,  6038,     3,  ...,     0,     0,     0],
        ...,
        [ 9302,  6038,   391,  ...,     0,     0,     0],
        [ 9302,  6038,   391,  ...,     0,     0,     0],
        [  205, 20931, 23936,  ...,    20,  2032,     1]]), 'attention_mask': tensor([[1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 1, 1, 1],
        [1, 1, 1,  ..., 0, 0, 0],
        ...,
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 1, 1, 1]])}

In [52]:
tokenized_train_notes

{'input_ids': tensor([[  262,     4,  4815,  ...,     0,     0,     0],
        [ 3388, 27447,  8015,  ...,     0,     0,     0],
        [    3,  7094, 22633,  ...,     0,     0,     0],
        ...,
        [    3, 12016,  5668,  ...,     0,     0,     0],
        [  262,     4,   188,  ...,     0,     0,     0],
        [ 3388, 27447,  8015,  ...,     0,     0,     0]]), 'attention_mask': tensor([[1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        ...,
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0]])}